In [ ]:
# validation.ipynb

# --- Step 1: Import Libraries and Load Data ---
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from pathlib import Path

print("--- Quality Control Lab ---")

# Load the master feature dataset
file_path = Path("../data/processed/master_m15_features.csv")
if not file_path.exists():
    print(f"ERROR: Dataset not found at {file_path}. Please run the Snakemake pipeline first.")
else:
    print(f"Loading dataset from: {file_path}")
    df = pd.read_csv(file_path, index_col='Datetime', parse_dates=True)
    print("Dataset loaded successfully. Shape:", df.shape)
    print("Columns:", df.columns.tolist())

# Set plot style for better readability
plt.style.use('seaborn-v0_8-darkgrid')
